This notebook compares the performance of the following classification models:
  1. Logistic Regression
  2. SVM
  3. Decision Tree
  4. Random Forest
  5. GaussianNB
  6. MultinomialNB
  7. BernoulliNB
  8. XG Boost

Each model has tuned hyperparameters, if applicable. The comparison uses GridSearchCV with 5 sampling splits. For now, the target was reduced to values 0 (Healthy) and 1 (Sick) rather than the expanded Sick classification.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from clean_data import HeartDisease

df = HeartDisease().modeMissing()
df = HeartDisease.reduceSick(df)
df.describe()

,age,sex,cp,testbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.663366,4.722772,0.458746
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.934375,1.938383,0.499120
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,1.000000


In [2]:
# Define features and target
features = df.drop('num', axis='columns')
target = df['num']

In [3]:
# Define models and their parameters
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from xgboost import XGBClassifier

models = {
  'logistic_regression': {
    'model': LogisticRegression(solver='liblinear'),
    'params': {
      'C': [1, 5, 10] # Default: 1
    }
  }, 
  'svm': {
    'model': SVC(),
    'params': {
      'C': [1, 5, 10], # Default: 1
      'kernel': ['rbf', 'linear', 'sigmoid'], # Default: rbf
      'gamma': ['scale', 'auto'] # Default: scale
    }
  },
  'decision_tree': {
    'model': DecisionTreeClassifier(),
    'params': {
      'criterion': ['gini', 'entropy'] # Default: gini
    }
  },
  'random_forest': {
    'model': RandomForestClassifier(),
    'params': {
      'n_estimators': [50, 100, 200], # Default: 100
      'criterion': ['gini', 'entropy'] # Default: gini
    }
  },
  'gaussian_nb': {
    'model': GaussianNB(),
    'params': {
      # No params
    }
  },
  'multinomial_nb': {
    'model': MultinomialNB(),
    'params': {
      # No params
    }
  },
  'bernoulli_nb': {
    'model': BernoulliNB(),
    'params': {
      # No params
    }
  },
  'xg_boost': {
    'model': XGBClassifier(),
    'params': {
      'n_estimators': [50, 100, 200],
    }
  }
}

In [4]:
# Use GridSearchCV for each model and report best scores
from sklearn.model_selection import GridSearchCV
scores = []
for model, props in models.items():
  gs = GridSearchCV(
    props['model'],
    props['params'],
    cv=5,
    return_train_score=False
  )
  gs.fit(features, target)
  scores.append({
    'model': model,
    'best_score': gs.best_score_,
    'best_params': gs.best_params_
  })

# Report scores as a dataframe
results = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
results

,model,best_score,best_params
0,logistic_regression,0.831585,{'C': 1}
1,svm,0.818361,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}"
2,decision_tree,0.765574,{'criterion': 'gini'}
3,random_forest,0.838087,"{'criterion': 'gini', 'n_estimators': 100}"
4,gaussian_nb,0.831639,{}
5,multinomial_nb,0.758962,{}
6,bernoulli_nb,0.798743,{}
7,xg_boost,0.801749,{'n_estimators': 100}
